# Atividade 2 - Visão Computacional

O entregável de toda esta atividade vai ser um código-fonte em C. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera


Detalhe como calculou $f$

Sendo $h_i$ a a distância entre os dois circulos na imagem, $h_f$ a distância entre os dois circulos na folha,
$f$, a distância focal, e $d$, a distância real entre a lente da câmera e a folha com os círculos, pode-se obter $f$ por meio da expressão:

$\frac{h_i}{f} = \frac {h_f}{d} $

In [1]:
hi = 545 - 126 #posiçao 'x' dos ciirculos --> obtem-se a distância entre um e outro em pixel
hf = 14
d = 15
f = (hi * d) / hf

print(f)

448.92857142857144


## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while(True):

    ret, frame = cap.read()
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_magenta, upper_magenta = aux.ranges('#FF000C')
    lower_cyan, upper_cyan = aux.ranges('#005DFF')
    
    mask1 = cv2.inRange(hsv, lower_magenta, upper_magenta)
    mask2 = cv2.inRange(hsv, lower_cyan, upper_cyan)
    #usando mask para identificar somente o que for magenta ou ciano
    mask = cv2.bitwise_or(mask1, mask2)
    
    blur = cv2.GaussianBlur(mask,(5,5),0)
    
    cv2.imshow('Magenta e Ciano', mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

hf = 14
f = 448.92857142857144

while(True):
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_magenta, upper_magenta = aux.ranges('#ff052a')
    lower_cyan, upper_cyan = aux.ranges('#0055FF')
    mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
    mask_cyan = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask = cv2.bitwise_or(mask_magenta, mask_cyan)
    
    #À partir da mask, buscar pelos maiores objetos identificados pelas cores magenta e ciano
    img_out, contornos, arvore = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((9, 9)))
    contornos_img = opening.copy()
    contornos_rgb = cv2.cvtColor(contornos_img, cv2.COLOR_GRAY2RGB)
    
    if len(contornos) > 0:
        maior1 = contornos[0]
        maior2 = contornos[0]
    maior_area = 0
    for c in contornos:
        area = cv2.contourArea(c)
        if area > maior_area:
            maior_area = area
            maior2 = maior1
            maior1 = c
    
    #obtendo ponto médio dos contornos para identificar o centro dos circulos magenta e ciano
    if len(contornos) > 0:
        x_m = [c[0][0] for c in maior1]
        y_m = [c[0][1] for c in maior1]
        mx_m = int(np.mean(x_m))
        my_m = int(np.mean(y_m))
        cv2.drawContours(contornos_rgb, [maior1], -1, [0, 255, 255], 5);
        cv2.circle(contornos_rgb,(mx_m,my_m),2,(0,0,255),3)
        x_c = [c[0][0] for c in maior2]
        y_c = [c[0][1] for c in maior2]
        mx_c = int(np.mean(x_c))
        my_c = int(np.mean(y_c))
        cv2.drawContours(contornos_rgb, [maior2], -1, [0, 255, 255], 5);
        cv2.circle(contornos_rgb,(mx_c,my_c),2,(0,0,255),3)
        
        #Fórmula matemática de distância de ponto a ponto.
        hi = ((mx_m - mx_c)**2 + (my_m - my_c)**2)**(1/2)
        
        if hi>0:
            d = f * hf / hi
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(contornos_rgb,'Distancia:{} cm'.format(int(d)),(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)
        
    cv2.imshow('Distancia dos centros', contornos_rgb)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<video width='360' height='288' controls src='Distancia_centros_teste.mp4'>

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

hf = 14
f = 448.92857142857144

while(True):
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_magenta, upper_magenta = aux.ranges('#ff052a')
    lower_cyan, upper_cyan = aux.ranges('#0055FF')
    mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
    mask_cyan = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask = cv2.bitwise_or(mask_magenta, mask_cyan)
    
    #À partir da mask, buscar pelos maiores objetos identificados pelas cores magenta e ciano
    img_out, contornos, arvore = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((9, 9)))
    contornos_img = opening.copy()
    contornos_rgb = cv2.cvtColor(contornos_img, cv2.COLOR_GRAY2RGB)
    
    if len(contornos) > 0:
        maior1 = contornos[0]
        maior2 = contornos[0]
    maior_area = 0
    for c in contornos:
        area = cv2.contourArea(c)
        if area > maior_area:
            maior_area = area
            maior2 = maior1
            maior1 = c
    
    #obtendo ponto médio dos contornos para identificar o centro dos circulos magenta e ciano
    if len(contornos) > 0:
        x_m = [c[0][0] for c in maior1]
        y_m = [c[0][1] for c in maior1]
        mx_m = int(np.mean(x_m))
        my_m = int(np.mean(y_m))
        cv2.drawContours(contornos_rgb, [maior1], -1, [0, 255, 255], 5);
        cv2.circle(contornos_rgb,(mx_m,my_m),2,(0,0,255),3)
        x_c = [c[0][0] for c in maior2]
        y_c = [c[0][1] for c in maior2]
        mx_c = int(np.mean(x_c))
        my_c = int(np.mean(y_c))
        cv2.drawContours(contornos_rgb, [maior2], -1, [0, 255, 255], 5);
        cv2.circle(contornos_rgb,(mx_c,my_c),2,(0,0,255),3)
        
        #desenhando a linha entre os centros dos círculos:
        cv2.line(contornos_rgb,(mx_c,my_c),(mx_m,my_m),(255,0,0),5)
        
        #calculando o coeficiente angular:
        if my_m > my_c:
            deltay = my_m - my_c
        else:
            deltay = my_c - my_m
            
        if mx_m > mx_c:
            deltax = mx_m - mx_c
        else:
            deltax = mx_c - mx_m
        if deltax > 0:
            m = deltay/deltax
            rad = np.arctan(m)
            graus = rad*180/np.pi

            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(contornos_rgb,'Inclinacao: {}'.format(int(graus)),(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)
        
    cv2.imshow('Angulo linha e horizontal', contornos_rgb)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<video width='360' height='288' controls src='Angulo_teste.mp4'>

## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

Já foi feito nos itens anteriores. Como pode ser observado nos arquivos de vídeo filmados, os contornos dos círculos são desenhados em amarelo.

In [6]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux

# If you want to open a video, just change this path
#cap = cv2.VideoCapture('hall_box_battery.mp4')

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []

    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here:
    # http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles = cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)
    
    a = bordas_color.copy()
    
    hsv = cv2.cvtColor(a, cv2.COLOR_BGR2HSV)
    lower_magenta, upper_magenta = aux.ranges('#ff052a')
    lower_cyan, upper_cyan = aux.ranges('#0055FF')
    mask_magenta = cv2.inRange(hsv, lower_magenta, upper_magenta)
    mask_cyan = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask = cv2.bitwise_or(mask_magenta, mask_cyan)

    # Display the resulting frame
    cv2.imshow('Detector de circulos', mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while(True):

    ret, frame = cap.read()
        
    MIN_MATCH_COUNT = 10

    img_original = cv2.imread('logoInsper.png',0)      # Gabarito / Imagem a procurar
    
    original_rgb = cv2.cvtColor(img_original, cv2.COLOR_GRAY2RGB)
    out = frame.copy()
    
    # Cria o detector SIFT
    sift = cv2.xfeatures2d.SIFT_create()

    # Encontra os pontos únicos (keypoints) nas duas imagems
    kp1, des1 = sift.detectAndCompute(img_original ,None)
    kp2, des2 = sift.detectAndCompute(frame,None)
    
    # Configurações do algoritmo FLANN que compara keypoints e ver correspondências - não se preocupe com isso
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # Configura o algoritmo de casamento de features que vê *como* o objeto que deve ser encontrado aparece na imagem
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)


    if len(good)>MIN_MATCH_COUNT:
        # Separa os bons matches na origem e no destino
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


        # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
        # Esta transformação é chamada de homografia 
        # Para saber mais veja 
        # https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()



        h,w = img_original.shape
        # Um retângulo com as dimensões da imagem original
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

        # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
        dst = cv2.perspectiveTransform(pts,M)


        # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
        img2b = cv2.polylines(out,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)

    else:
        matchesMask = None

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matchesMask, # draw only inliers
                       flags = 2)
    
    cv2.imshow('Identifica logo', out)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<video width='360' height='288' controls src='Logo_insper.mp4'>